 <div style="text-align:center;">
   <span style="color:green; font-size:larger; font-weight:bold;">Exploratory Data Analysis (EDA)</span><br><br>
</div>


<span style="color:green; font-size:larger; font-weight:bold;">INTRODUCTION</span>

Dans cette partie, nous nous intéressons à l'analyse descriptive de la base de données "finale_merged" obtenue suite au nettoyage des bases de données initiales du fichier "Préparation_données.ipynb"